To obtain an atomistic structure from coarse grained simulations we need a CG structure and a topology file for the atomistic system.

In [1]:
import running_rabbit as rr

In [2]:
# input
cg_structure = "2xUBQ_cg.pdb"
aa_topology  = "2xUBQ_aa.top"

# output
aa_structure_raw = "2xUBQ_aa_raw.pdb"    # initial structure after running backward.py
aa_structure     = "2xUBQ_aa.pdb"        # final structure after energy min.
path_em          = "./em_after_backmap"

In [3]:
# HOTFIX, convert into package at some point?
backward = "python /home/andrejb/Software/Backward/backward.py"

In [4]:
# perform backmapping
!{backward} -f {cg_structure} -p {aa_topology} -o {aa_structure_raw} -to gromos -kick 0.05

Residues defined for transformation from martini to gromos:
['POPC', 'ILE', 'POPE', 'DOPC', 'POPS', 'GLY', 'ALA', 'GLUC', 'CYS', 'HIS', 'GLN', 'SEP', 'PRO', 'CHOL', 'GLU', 'AOT', 'CL4M', 'CL4O', 'ASN', 'DMPC', 'HEP', 'VAL', 'DPG1', 'DOPE', 'DPPC', 'TRE', 'THR', 'ASP', 'TRP', 'SER', 'DSPC', 'LYS', 'PHE', 'MGDG', 'MET', 'LEU', 'ARG', 'TYR']
chiral ['CB', 'CA', 'N', 'C']
chiral ['CB', 'CA', 'N', 'C']
out ['NE2', 'CD', 'OE1', 'CG']
out ['HE21', 'CD', 'OE1', 'CG']
out ['HE22', 'CD', 'OE1', 'CG']
out ['OE1', 'CD', 'NE2', 'CG']
chiral ['CB', 'CA', 'N', 'C']
chiral ['CB', 'CA', 'N', 'C']
out ['P', 'CD1', 'CD2', 'CZ']
out ['Q', 'CD2', 'CD1', 'CZ']
out ['R', 'CZ', 'CD1', 'CD2']
trans ['HD1', 'P', 'CE1', 'R']
trans ['HD2', 'Q', 'CE2', 'R']
trans ['HE1', 'CE1', 'P', 'CG']
trans ['HE2', 'CE2', 'Q', 'CG']
trans ['HZ', 'R', 'CE1', 'P']
out ['CD1', 'CE1', 'HE1', 'R']
out ['CD2', 'CE2', 'HE2', 'R']
out ['CZ', 'CE1', 'HE1', 'P']
chiral ['CB', 'CA', 'N', 'C']
chiral ['CB', 'CA', 'N', 'C']
chiral ['CB', '

The atomistic structure after backmapping is highly unphysical and has to be energy minimized to obtain a proper structure. Therefore, we perform energy minimization in three steps:
1. Backbone position constrained, small em step, higher force threshold, no bond constraints 
2. Backbone position constrained, bigger em step, lower force threshold, no bond constraints
3. no position constraints, bigger em step, lower force threshold, bonds constrained with LINKS 

In [5]:
# note: add running rabbit to setup simulations, therefore prepare rr template for EQ of backmapped structures
rabbit              = rr.Rabbit(ff="gromos54a7", template_name="backmap_em")
rabbit.structure    = aa_structure_raw
rabbit.topology     = aa_topology
rabbit.destination  = path_em
rabbit.run()

True

In [ ]:
# run EM
!cd em_after_backmap && bash setup

In [ ]:
# copy final structure
!cp {path_em}/eq5.pdb {aa_structure}

In [ ]:
# note: some atoms are not in the first unit cell after mackmapping
# resolve by:
!gmx editconf -f {aa_structure} -o {aa_structure} -pbc yes